## Label graph data with optimal solution
1. Contextualization: Convert numerical value to text-based attributes

2. Optimization: Use MIP to get the optimal solution

In [15]:
import numpy as np
from tqdm import tqdm
import json
import sys
import os
from gurobipy import Model, GRB
import pandas as pd

sys.path.append('src')
# sys.path.append('gnn')
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

In [3]:
env_config_name = "large_graph_test"
save_datapath = f"src/gnn/gnn_dataset"

In [26]:
with open(f"gnn_dataset/gnn_data_Env({env_config_name}).json", 'r') as f:
    dataset = json.load(f)

### Optimization

In [27]:
dataset[0].keys()

dict_keys(['stage_id', 'demands', 'inventory', 'lead_times', 'prod_capacity', 'sale_price', 'order_costs', 'prod_cost', 'holding_cost', 'backlog_cost', 'fulfilled_rate', 'orders', 'profit', 'sales_t', 'inventory_t', 'arrival_record'])

In [ ]:
df = pd.DataFrame(
    {
        "demands": [sum(data) for data in dataset[0]['demands']][:-10],
        "orders": [sum(data) for data in dataset[0]['orders']],
        "sales t": dataset[0]['sales_t'][:-10],
        'inventory': dataset[0]['inventory_t'][:-10],
        "profits": dataset[0]['profit'], 

    }
)


In [33]:
df

,demands,orders,sales t,profits,inventory
0,6,30.0,6.0,74492.0,11.0
1,4,16.0,4.0,74492.0,5.0
2,12,22.0,12.0,74492.0,12.0
3,9,13.0,9.0,74492.0,9.0
4,8,12.0,8.0,74492.0,8.0
5,9,14.0,9.0,74492.0,9.0
6,10,19.0,10.0,74492.0,10.0
7,8,15.0,8.0,74492.0,8.0
8,13,18.0,13.0,74492.0,13.0
9,13,17.0,13.0,74492.0,13.0


### Contextualization

In [ ]:
num_data = len(dataset)
num_period = len(dataset[0]['demands'])
num_agents_per_stage = len(dataset[0]["lead_times"])

In [ ]:
# num_stages = env_config["num_stages"]
contextualized_dataset = []

for data in tqdm(dataset):

    stage_id = data["stage_id"]
    sale_price = data["sale_price"]
    inventory = data['inventory']
    prod_capacity = data['prod_capacity']

    # Node attributes
    inventory_text = f"Your current inventory is {inventory} unit(s)"
    prod_capacity = f"Your production capacity is {prod_capacity} unit(s)"
    order_costs = data['order_costs']
    ordercost_text_list = []
    for i in range(num_agents_per_stage):
        supplier_name = f"stage_{stage_id+1}_agent_{i}"
        ordercost_text_list.append(f"{supplier_name} has product price {order_costs[i]} unit(s)")

    # Edge attributes
    lead_times = data['lead_times']
    leadtime_text_list = []
    for i in range(num_agents_per_stage):
        supplier_name = f'stage_{stage_id+1}_agent_{i}'
        leadtime_text_list.append(f"You have lead time of {lead_times[i]} day(s) to {supplier_name}")
    
    demands_text_list = [fds]
    for t in range(num_period):
        demands = data['demands'][t]
        for i in range(num_agents_per_stage):
            if demands[i] == 0: # not a customer
                continue
            customer_name = f'stage_{stage_id-1}_agent_{i}'
            demands_text_list.append(f"{customer_name} requests {demands[i]} unit(s) of product from you")

        cdata = {}
        cdata['nodes'] = [inventory_text, prod_capacity] + ordercost_text_list
        cdata['edges'] = leadtime_text_list + demands_text_list
        contextualized_dataset.append(cdata)
# "demand": demand_seq, 
# "inventory": init_inventories[i%num_agents].tolist(), 
# "lead_times": lead_times[stage_id+1, i%num_agents_per_stage, :].tolist(), 
# "prod_capacity": prod_capacities[i%num_agents].tolist(),
# "sale_price": sale_prices[i%num_agents].tolist(),
# "order_costs": order_costs[(stage_id+1)*num_agents_per_stage: (stage_id+2)*num_agents_per_stage].tolist(), 
# "prod_costs": prod_costs[i%num_agents].tolist(), 
# "holding_cost": holding_costs[i%num_agents].tolist(),
# "backlog_cost": backlog_costs[i%num_agents].tolist(),
# "fulfilled_rate": fulfilled_rate.tolist(), 

In [ ]:
# contextualized_dataset[0]